# What have we done so far
### De-duped parcel_id's
### Selected predictive features
### Cleaned the data (ONLY A BIT)

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

In [2]:
# Load the joined dataset
df = pd.read_csv("../../data/data-jam-joined.csv", low_memory=False)
print(f"Original dataframe shape: {df.shape}")

Original dataframe shape: (511687, 94)


### Predictive columns:
- year_built, tax_status, assessed_value, is_improved, total_floor_area, property_class_desc, IS_OCCUPIED (field), FACADE_SIDING_CONDITION, FIRE_DAMAGE_CONDITION
- unit_cleared_at (dependent on time), permit_type (trade permits, date), issued_date (when permit issued), 
- VACANT (USPS), MAILER_EXPORT_DATE (when we determined they were vacant), account_active_code (water status, might be mangled)
- pct_pre_claimed, total_acreage
- local_historic_district: predictive, but discriminatory.
- SURVEYOR_NOTES (free form text we can topic model against)


### Predictive, but we won't have them at "prediction time":
- FACADE_SIDING_CONDITION, FIRE_DAMAGE_CONDITION, ROOF_CONDITION, OPENINGS_CONDITION, IS_OPEN_TO_TREPASS, PORCH_STEPS_CONDITION

### Helper columns: (filters, id's)
- PARCEL_ID_x, DATE_SURVEYED(filter to last surveyed), FIELD_DETERMINATION(blight labels)
- HAS_STRUCTURE (throw it out if it's false)

In [3]:

selected_columns = [
    'PARCEL_ID_x',  # Single parcel ID
    'year_built_x', 'tax_status_x',
    'assessed_value', 'is_improved_x', 'total_floor_area_x',
    'property_class_desc', 'DATE_SURVEYED',
    'FIELD_DETERMINATION', 'SURVEYOR_NOTES', 'HAS_STRUCTURE', 'IS_OCCUPIED',
    'unit_cleared_at', 'address_id', 
    'record_id', 'permit_type', 'issued_date', 'address_id_trd',
    'VACANT', 'MAILER_EXPORT_DATE',
    'address_number', 'address_street_name',
    'account_active_code', 'pct_pre_claimed', 'total_acreage', 'local_historic_district', 
]


tight_df = df[selected_columns]

In [4]:
# Display first few rows of the tight dataframe
tight_df.head()

,PARCEL_ID_x,year_built_x,tax_status_x,assessed_value,is_improved_x,total_floor_area_x,property_class_desc,DATE_SURVEYED,FIELD_DETERMINATION,SURVEYOR_NOTES,HAS_STRUCTURE,IS_OCCUPIED,unit_cleared_at,address_id,record_id,permit_type,issued_date,address_id_trd,VACANT,MAILER_EXPORT_DATE,address_number,address_street_name,account_active_code,pct_pre_claimed,total_acreage,local_historic_district
0,08010116.,1927.0,TAXABLE,50400.0,1.0,2600.0,RESIDENTIAL-IMPROVED,2024-07-03 00:00:00,Noticeable Evidence of Blight,Front entrance boarded up.,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Y,2021-10-01,16191.0,BAYLIS ST,ACTIVE,NaN,NaN,NaN
1,08010116.,1927.0,TAXABLE,50400.0,1.0,2600.0,RESIDENTIAL-IMPROVED,2024-07-03 00:00:00,Noticeable Evidence of Blight,Front entrance boarded up.,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Y,2020-05-01,16191.0,BAYLIS ST,ACTIVE,NaN,NaN,NaN
2,08010116.,1927.0,TAXABLE,50400.0,1.0,2600.0,RESIDENTIAL-IMPROVED,2024-07-03 00:00:00,Noticeable Evidence of Blight,Front entrance boarded up.,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Y,2022-10-01,16191.0,BAYLIS ST,ACTIVE,NaN,NaN,NaN
3,08010116.,1927.0,TAXABLE,50400.0,1.0,2600.0,RESIDENTIAL-IMPROVED,2024-07-03 00:00:00,Noticeable Evidence of Blight,Front entrance boarded up.,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,N,2023-11-01,16191.0,BAYLIS ST,ACTIVE,NaN,NaN,NaN
4,08010116.,1927.0,TAXABLE,50400.0,1.0,2600.0,RESIDENTIAL-IMPROVED,2024-07-03 00:00:00,Noticeable Evidence of Blight,Front entrance boarded up.,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Y,2023-11-01,16191.0,BAYLIS ST,ACTIVE,NaN,NaN,NaN


In [5]:
tight_df['HAS_STRUCTURE'].value_counts()

HAS_STRUCTURE
1.0    446954
0.0      4241
Name: count, dtype: int64

In [6]:
tight_df = tight_df[tight_df['HAS_STRUCTURE'] == 1.0]
tight_df = tight_df.drop(columns=['HAS_STRUCTURE'])

In [7]:
tight_df.shape

(446954, 25)

In [8]:
tight_df.columns

Index(['PARCEL_ID_x', 'year_built_x', 'tax_status_x', 'assessed_value',
       'is_improved_x', 'total_floor_area_x', 'property_class_desc',
       'DATE_SURVEYED', 'FIELD_DETERMINATION', 'SURVEYOR_NOTES', 'IS_OCCUPIED',
       'unit_cleared_at', 'address_id', 'record_id', 'permit_type',
       'issued_date', 'address_id_trd', 'VACANT', 'MAILER_EXPORT_DATE',
       'address_number', 'address_street_name', 'account_active_code',
       'pct_pre_claimed', 'total_acreage', 'local_historic_district'],
      dtype='object')

In [9]:
# TODO: Keep the latest survey.

# Drop erroneous parcel's that have null join's amounting to 12,742 rows.
tight_df = tight_df[~tight_df['PARCEL_ID_x'].isin(['22118326.',
  '02001944.', '22087886-7',
  '22087888.', '08007564.',
  '08010001.', '09006208.',
  '01000670-1', '17007609.',
  '18011974.'])]

In [10]:
tight_df.shape

(319534, 25)

In [11]:
tight_df.columns

Index(['PARCEL_ID_x', 'year_built_x', 'tax_status_x', 'assessed_value',
       'is_improved_x', 'total_floor_area_x', 'property_class_desc',
       'DATE_SURVEYED', 'FIELD_DETERMINATION', 'SURVEYOR_NOTES', 'IS_OCCUPIED',
       'unit_cleared_at', 'address_id', 'record_id', 'permit_type',
       'issued_date', 'address_id_trd', 'VACANT', 'MAILER_EXPORT_DATE',
       'address_number', 'address_street_name', 'account_active_code',
       'pct_pre_claimed', 'total_acreage', 'local_historic_district'],
      dtype='object')

In [12]:
tight_df.to_csv("data-jam-joined-cleaner-08042025.csv")

In [14]:
# CV model does roof, opening (missing a wall)

# Roof: 67.72% accuracy
# Wall: 57%
# Opening: 67%

In [ ]:
# Define predictive columns
predictive_columns = [
    'year_built_x', 'tax_status_x',
    'assessed_value', 'is_improved_x',
'total_floor_area_x', 'DATE_SURVEYED',
    'property_class_desc',
    'SURVEYOR_NOTES', 'IS_OCCUPIED',
    'permit_type',
    'VACANT',
    'account_active_code', 'pct_pre_claimed',
'total_acreage', 'local_historic_district',
]

df_features = tight_df[['PARCEL_ID_x'] +
  predictive_columns]

# Define the valid blight labels
valid_labels = {
    'Vacant (Not Blighted)': 0,
    'No Action (Salvage)': 0,
    'Noticeable Evidence of Blight': 1,
    'Significant Evidence of Blight': 2,
    'Extreme Evidence of Blight': 3
}

# Filter to only keep rows with valid 
df_labels = tight_df[tight_df['FIELD_DETERMINATION'].isin(valid_labels.keys())][['PARCEL_ID_x','FIELD_DETERMINATION']].copy()

# Create numeric label column
df_labels['label'] = df_labels['FIELD_DETERMINATION'].map(valid_labels)

# Create string label column (keeping original names)
df_labels['label_str'] = df_labels['FIELD_DETERMINATION']

# Drop the original FIELD_DETERMINATION column if you 
df_labels = df_labels[['PARCEL_ID_x', 'label', 'label_str']]

In [25]:
df_labels['label'].value_counts()

label
1    171274
0     77987
2     60436
3      9563
Name: count, dtype: int64

In [ ]:
df_features.to_csv("../../training_data/08042025-cleaned-features.csv")

In [ ]:
df_labels.to_csv("../../training_data/08042025-cleaned-labels.csv")

In [29]:
df_features.shape

(319534, 15)

In [30]:
df_labels.shape

(319260, 3)